In [12]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [15]:
file_path = '/input/eggnog/Actinokineospora/concatenated_annotations.csv'
concatenated_df = pd.read_csv(file_path)

In [34]:
concatenated_df.head()

,Locus_ID,seed_ortholog,evalue,score,eggNOG_OGs,max_annot_lvl,COG_category,Description,Preferred_name,GOs,...,KEGG_Pathway,KEGG_Module,KEGG_Reaction,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,PFAMs,Strain
0,JOD54_RS00005,1535422.ND16A_1512,1.300000e-99,308.0,"COG0520@1|root,COG0520@2|Bacteria,1R4SM@1224|P...",1236|Gammaproteobacteria,M,Class V aminotransferase,-,-,...,"ko00311,ko01100,ko01130,map00311,map01100,map0...",M00673,R04147,RC00302,"ko00000,ko00001,ko00002,ko01000",-,-,-,Aminotran_5,GCF_016907695.1
1,JOD54_RS00010,1120934.KB894437_gene4492,7.160000e-86,258.0,"COG1595@1|root,COG1595@2|Bacteria,2HGII@201174...",201174|Actinobacteria,K,"Sigma-70, region 4",-,-,...,-,-,-,-,"ko00000,ko03021",-,-,-,"Sigma70_r2,Sigma70_r4_2",GCF_016907695.1
2,JOD54_RS00015,1068980.ARVW01000001_gene2852,3.050000e-92,279.0,"COG0350@1|root,COG2169@1|root,COG0350@2|Bacter...",201174|Actinobacteria,FL,Metal binding domain of Ada,-,-,...,-,-,-,-,"ko00000,ko01000,ko03400",-,-,-,"Ada_Zn_binding,DNA_binding_1",GCF_016907695.1
3,JOD54_RS00020,1120934.KB894437_gene4494,3.590000e-67,207.0,"COG0683@1|root,COG0683@2|Bacteria,2GT1Z@201174...",201174|Actinobacteria,E,leucine binding,-,-,...,-,-,-,-,-,-,-,-,"Ada_Zn_binding,Peripla_BP_6",GCF_016907695.1
4,JOD54_RS00025,1125971.ASJB01000083_gene1766,3.500000e-108,316.0,"COG3145@1|root,COG3145@2|Bacteria,2GMSV@201174...",201174|Actinobacteria,L,2OG-Fe(II) oxygenase superfamily,alkB,-,...,-,-,-,-,"ko00000,ko01000,ko03400",-,-,-,2OG-FeII_Oxy_2,GCF_016907695.1


In [17]:
# Load the dataframe 
file_path = '/results/Actinokineospora/cog_categories_summary_tta.csv'
cog_all_tta = pd.read_csv(file_path)

In [36]:
cog_all_tta.head()

,-,BDLTU,C,CE,CG,CH,CO,D,DL,DTZ,...,NU,O,P,Q,QT,S,T,U,Unknown,V
GCF_ID,,,,,,,,,,,,,,,,,,,,,
GCF_900111175.1,60,0,16,1,0,3,0,2,0,0,...,3,19,20,39,1,102,37,3,25,20
GCF_004362515.1,45,1,11,0,0,1,0,4,0,0,...,3,12,15,20,2,84,20,2,28,10
GCF_024760565.1,20,1,6,0,0,2,1,1,0,0,...,1,6,5,7,1,48,10,2,15,6
GCF_900101685.1,39,0,10,0,1,2,0,7,0,1,...,1,7,8,23,2,98,17,3,25,12
GCF_014648415.1,26,1,7,0,0,0,1,0,0,0,...,2,5,7,7,1,49,14,3,24,8


In [18]:
# Load dataframe with TTA containing genes
file_path = '/results/Actinokineospora/all_first_TTA_Actinokineospora.csv'
TTA_df = pd.read_csv(file_path)

In [19]:
cog_df = pd.merge(UUA_df, concatenated_df, on = 'Locus_ID', how = 'left')

In [20]:
cog_df = cog_df[cog_df["Relative_Position"] <= 10]

In [21]:
def count_cog_categories_per_strain(df, cog_column="COG_category", strain_column="Strain"):
    """
    Count occurrences of each COG category for each strain.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing the data.
        cog_column (str): Name of the column containing COG categories.
        strain_column (str): Name of the column containing strain names.
    
    Returns:
        pd.DataFrame: Summary DataFrame with counts of each COG category per strain.
    """
    if cog_column not in df.columns or strain_column not in df.columns:
        raise ValueError(f"'{cog_column}' or '{strain_column}' column not found in the DataFrame.")

    # Group by strain and COG category and count occurrences
    summary_df = (
        df.groupby([strain_column, cog_column])
        .size()
        .reset_index(name="Count")
        .pivot(index=strain_column, columns=cog_column, values="Count")
        .fillna(0)  # Fill missing values with 0
        .astype(int)  # Convert counts to integers
    )
    
    return summary_df

# Assuming the DataFrame is named `df`
cog_summary_tta_p0 = count_cog_categories_per_strain(cog_df, cog_column="COG_category", strain_column="Strain")

# Save the summary of number of genes in each COG category where TTA is in the first 10% within the gene length to a CSV file
output_file = '/results/Actinokineospora/cog_categories_summary_position_0.csv'
cog_summary_tta_p0.to_csv(output_file)